In [ ]:
def get_sorted_pxls(img, segmentation, segment):
    """
    img: input image
    segmentation: segmentation image of given input image
    segment: segment of interest
    Returns an array area(segment)-by-3, first two columns are indeces, third column is pixel intensities
    """
    pxIdxVals=np.zeros((np.sum(segmentation==segment),3))
    pxIdxVals[:,:2]=np.array(list(zip(*np.where(segmentation==segment))))
    pxIdxVals[:,2]=img[np.where(segmentation==segment)]
    pxIdxValsSorted=pxIdxVals[pxIdxVals[:,2].argsort()]
    
    return pxIdxValsSorted

In [ ]:
def img_zscore(img,baseline):
    """
    Returns z-score based on baseline mean and std
    """
    return (img-np.mean(baseline))/np.std(baseline)

def minima_pixel(img, index, minimaTmp, minimIdx):
    if minimaTmp>img[index[0],index[1]]:
        minimaTmp=img[index[0],index[1]]
        minimIdx=index
    return minimaTmp, minimIdx

In [ ]:
def convert_grayscale(img):
    img = ((img - img.min()) * (1/(img.max() - img.min()) * 255)).astype('uint8')
    return img

In [ ]:
def twoD_Gaussian(xy, amplitude, xo, yo, sigma_x, sigma_y, theta, offset):
    x, y = xy
    xo = float(xo)
    yo = float(yo)    
    a = (np.cos(theta)**2)/(2*sigma_x**2) + (np.sin(theta)**2)/(2*sigma_y**2)
    b = -(np.sin(2*theta))/(4*sigma_x**2) + (np.sin(2*theta))/(4*sigma_y**2)
    c = (np.sin(theta)**2)/(2*sigma_x**2) + (np.cos(theta)**2)/(2*sigma_y**2)
    g = offset + amplitude*np.exp( - (a*((x-xo)**2) + 2*b*(x-xo)*(y-yo) 
                            + c*((y-yo)**2)))
    return g.ravel()

def fit_2d_gaussian(xy, data, initial_guess):
    x,y=xy
    popt, pcov = scipy.optimize.curve_fit(twoD_Gaussian, (x, y), data.ravel(), p0=initial_guess)
    data_fitted = twoD_Gaussian((x, y), *popt)
    
    return data_fitted, popt, pcov

# Filters

In [ ]:
def gaussian_blur(img, kernel_size=5, sigma=0):
    blur = cv2.GaussianBlur(img,(kernel_size,kernel_size),sigma)
    return blur


def medfilt_vol(data, kernel=3):
    data_filt=data
    for i in range(0, len(data[0,0,:])):
        img=data[:,:,i]
        img=signal.medfilt2d(img, kernel)
        data_filt[:,:,i]=img
    return data_filt


def std_filt(data, alpha=1.5):
    for idx in range(len(data[0,0,:])):
        img=data[:,:,idx]
        img=signal.medfilt2d(img)
        img=img*(img>alpha*np.std(img))
        data[:,:,idx]=img
        
    return data

# Biascorrection

In [ ]:
def biascorrect_data(data, axis=2):
    if axis==2:
        for i in range(0, len(data[0,0,:])):
            test_img=data[:,:,i]
            plt.imsave('n4test.png',test_img)
            n4_test_img=n4bias(['n4test.png', "N4_corrected.nrrd", 1])
            corrected=convert_grayscale(sitk.GetArrayFromImage(n4_test_img['corrected_image']))
    #         plt.imshow(corrected)
    #         plt.show()
            data[:,:,i]=corrected
    elif axis==1:
        for i in range(0, len(data[0,:,0])):
            test_img=data[:,i,:]
            plt.imsave('n4test.png',test_img)
            n4_test_img=n4bias(['n4test.png', "N4_corrected.nrrd", 1])
            corrected=convert_grayscale(sitk.GetArrayFromImage(n4_test_img['corrected_image']))
    #         plt.imshow(corrected)
    #         plt.show()
            data[:,i,:]=corrected
        
    elif axis==0:
#         for i in range(0, len(data[0,:,0])):
        test_img=data
        plt.imsave('n4test.png',test_img)
        n4_test_img=n4bias(['n4test.png', "N4_corrected.nrrd", 1])
        corrected=convert_grayscale(sitk.GetArrayFromImage(n4_test_img['corrected_image']))
#         plt.imshow(corrected)
#         plt.show()
        data=corrected
        
    return data

In [ ]:
def n4bias(args):
    if len(args) < 2:
        print(
            "Usage: N4BiasFieldCorrection inputImage "
            + "outputImage [shrinkFactor] [maskImage] [numberOfIterations] "
            + "[numberOfFittingLevels]"
        )
        sys.exit(1)

    inputImage = sitk.ReadImage(args[0], sitk.sitkFloat32)
#     inputImage=args[0]
    image = inputImage

    if len(args) > 4:
        maskImage = sitk.ReadImage(args[4], sitk.sitkUInt8)
    else:
        maskImage = sitk.OtsuThreshold(inputImage, 0, 1, 200)

    shrinkFactor = 1
    if len(args) > 3:
        shrinkFactor = int(args[2])
        if shrinkFactor > 1:
            image = sitk.Shrink(
                inputImage, [shrinkFactor] * inputImage.GetDimension()
            )
            maskImage = sitk.Shrink(
                maskImage, [shrinkFactor] * inputImage.GetDimension()
            )

    corrector = sitk.N4BiasFieldCorrectionImageFilter()

    numberFittingLevels = 4

    if len(args) > 6:
        numberFittingLevels = int(args[6])

    if len(args) > 5:
        corrector.SetMaximumNumberOfIterations(
            [int(args[5])] * numberFittingLevels
        )

    corrected_image = corrector.Execute(image, maskImage)
    log_bias_field = corrector.GetLogBiasFieldAsImage(inputImage)
    corrected_image_full_resolution = inputImage / sitk.Exp(log_bias_field)


    return_images = {"input_image": inputImage,
                     "mask_image": maskImage,
                     "log_bias_field": log_bias_field,
                     "corrected_image": corrected_image}
    return return_images